### (Initializing)

In [2]:
import numpy as np

# Formulation of Functions
From "Fundamental of Machine Component Design", Section 12

Note: 

(F-32)*5/9 = C

6.89 * microreyn = milli-PascalSec  --> 6.89 * microreyn /1000 = PascalSec --> 6.89 * 1000 reyn = PascalSec

## Viscosity Functions
From "Fundamentals of Machine Component Design" Section 12, pp 633-635
Reference TBL12-1 and others in the section


In [43]:
# It shows empiral formulas for the basic SAE oils per Seireg
def Visc_SAE_Seireg(Temp, Grade, Metric = False):
    if (Metric):
        Temp = (Temp * 9/5) + 32 # C to F
    match Grade:
        case 10:
            m0 = 0.0158E-6
            b = 1157.5
        case 20:
            m0 = 0.0136E-6
            b = 1271.6
        case 30:
            m0 = 0.0141E-6
            b = 1360.0
        case 40:
            m0 = 0.0121E-6
            b = 1474.4
        case 50:
            m0 = 0.0170E-6
            b = 1509.6
        case 60:
            m0 = 0.0187E-6
            b = 1564.0
    if (not Metric):
        return m0 * np.e**(b / (Temp + 95))    
    else:
        return m0 * np.e**(b / (Temp + 95)) * 6.89 * 1000 # convert back to Metric

# Exponential Fit of the Below Data Source, as found in the GDrive
# https://wiki.anton-paar.com/us-en/engine-oil/
def Visc_SAE_Inter(Temp, Grade, Metric = False):
    match Grade:
        case "30":
            if (Metric): mu = 243.9 * Temp**(-2.24)
            else: mu = 119.4 * Temp**(-3.48)
        case "10W-40":
            if (Metric): 
                if(Temp > 10): # very inaccurate fit below this
                    mu = 19.6 * Temp**(-1.56)
                else: 
                    print("Outside of Interpolation acceptability range (Metric)")
                    return None
            else: 
                if(Temp > 50): # very inaccurate fit below this
                    mu = 0.348 * Temp**(-2.25)
                else: 
                    print("Outside of Interpolation acceptability range (Imperial)")
                    return None
    return mu

            

## Factors & Coefficients
Sommerfeld#, Friction and Flow Factors, Temperature Factors & Delta

In [44]:
# Interchange between Metric and Imperial. "Metric" is if the input is Celsius to begin with, and will convert to F
def ConvertTemp(Temp, Metric = False):
    if (Metric):
        return (Temp * 9/5) + 32
    else:
        return (Temp - 32) * 5/9

In [45]:
# Sommerfeld Calculation for Naturally Lubricated Journal Bearings
def Sommerfeld(mu, r, c, N, p, echo = False):
    S = (float(r)/float(c))**2 * mu * float(N) / float(p) 
    if(echo): print("Sommerfeld #: " + str(S))
    return S 

In [46]:
# Calculation of temperature change via the Temperature Factor (see below)
def TempDelta(Tval, p, Metric = False, iter = 0):
    if (Metric): delT = Tval * p / 0.120 * 1E-6 # for Metric
    else: delT = Tval * p / 9.7 # for Imperial

    if iter != 0:
        print("Temperature Delta (" + str(iter) + "): " + str(delT))    
    return delT

### Friction Factor Interpolating Function

In [47]:
# Frictional Coefficent r/c * f calculation
#   Polynomial Fit to readings from Fig 12-18. 
#   4th order chosen for Infinite Case
#   Length l need not be provided for infinite case
def FricFactor(S, mu, N, p, r, c, l = None, inf = True, echo = True):
    d = 2*r
    if (inf):
        if (S > 0.5): 
            if (echo): 
                print("Petroff's Equation for Extrapolation")
            f = 2 * np.pi**2 * mu * N / P * r 
        else:
            A = 0.407
            B = 22.7
            C = -86.8
            D = 310
            E = -310
            f = A + B * S + C * S**2 + D * S**3 + E * S**4
    elif (abs(l - d) < .001): #L/D = 1
        if (S < .05):
            A = .0797
            B = 92.3
            C = -2798
            D = 47909
            E = -300641
            f = A + B * S + C * S**2 + D * S**3 + E * S**4
        elif (S < 1):
            A = .567
            B = 21.9
            C = -10.3
            D = 9.17
            E = -2.41
            f = A + B * S + C * S**2 + D * S**3 + E * S**4
        else:
            if (echo): 
                print("Petroff's Equation for Extrapolation")
            f = 2 * np.pi**2 * mu * N / P * r 
    elif (abs(l/d - 1/2) < .001): # L/D = 1/2
        if (S < .025): # Rsqr = .998 for S = .002 -> .035 
            A = .159
            B = 67
            C = 1103
            D = -122837
            E = 2.14E6
            f = A + B * S + C * S**2 + D * S**3 + E * S**4
        elif (S < 1):# Rsqr = 1 for S = .002 -> 1.2
            A = .555
            B = 31.7
            C = -37.2
            D = 41.1
            E = -15.4
            f = A + B * S + C * S**2 + D * S**3 + E * S**4
        else:
            if (echo): print("Petroff's Equation for Extrapolation")
            f = 2 * np.pi**2 * mu * N / P * r
    elif (abs(l/d - 1/4) < .001): # L/D = 1/4
        if (S < .03): # Rsqr = .999 for S = .002 -> 0.035
            A = .0224
            B = 117
            C = -3389
            D = 56915
            E = -259109
            f = A + B * S + C * S**2 + D * S**3 + E * S**4
        elif (S < 4): #Rsqr = 1 for S = .002 ->  4
            A = .805
            B = 30
            C = -10.1
            D = 3.52
            E = -.425
            f = A + B * S + C * S**2 + D * S**3 + E * S**4
        else:
            if (echo): 
                print("Petroff's Equation for Extrapolation")
            f = 2 * np.pi**2 * mu * N / P * r 
    else: # interpolation formula
        if(echo): print("Interpolated Friction Factor") # Reference EQ 12-16

        yinf = FricFactor(S = S, mu = mu, N = N, p = p, r = r, c = c, inf = True)
        c_yinf = -1/8 * (1 - l/d) * (1 - 2 * l/d) * (1 - 4 * l/d)

        y1 = FricFactor(S = S, mu = mu, N = N, p = p, r = r, c = c, l = d)
        c_y1 = 1/3 * (1 - 2 * l/d) * (1 - 4 * l/d)

        y12 = FricFactor(S = S, mu = mu, N = N, p = p, r = r, c = c, l = d/2)
        c_y12 = -1 /  4 * (1 - l/d) * (1 - 4 * l/d)
        
        y14 = FricFactor(S = S, mu = mu, N = N, p = p, r = r, c = c, l = d/4)
        c_y14 = 1 / 24 * (1 - l/d) * (1 - 2 * l/d)

        f = (l/d)**(-3) * (c_yinf * yinf + c_y1 * y1 + c_y12 * y12 + c_y14 * y14)      
    return f  

### Temp Factor Interpolating Function

In [48]:

# Temperature Factor Evaluation for Infinite L/D ratio only
#   Simplification of formula 12-15
def TempFactorINF(S, mu, N, p, r, c):
    frc = FricFactor(S = S, mu = mu, N = N, p = p, r = r, c = c, inf = True, echo = True)
    Qrcnl = FlowFactor(S = S, inf = True)
    return frc / Qrcnl

# Calculation of Temperature Factor for limited L/D ratio
#   Reference Fig 12-24, EQN 12-16
#   Requires inputs to mu, N, p, c to do interpolation
def TempFactor(S, l, d, mu = None, N = None, p = None, c = None, echo = False):
    r = d / 2
    if(abs(l - d) < .001):
        factT = .349109 + 6.00940 * S + .047467 * S**2
    elif(abs(l/d - .5) < .001):
        factT = .394552 + 6.392527 * S - .036013 * S**2
    elif(abs(l/d - .25) < .001):
        factT = .933828 + 6.437512 * S - .011048 * S**2
    else: 
        if(echo): print("Interpolated Temp Factor") # Reference EQ 12-16

        yinf = TempFactorINF(S = S, mu = mu, N = N, p = p, r = r, c = c)
        c_yinf = -1/8 * (1 - l/d) * (1 - 2 * l/d) * (1 - 4 * l/d)

        y1 = TempFactor(S = S, l = 1, d = 1)
        c_y1 = 1/3 * (1 - 2 * l/d) * (1 - 4 * l/d)

        y12 = TempFactor(S = S, l = 1, d = 2)
        c_y12 = -1 /  4 * (1 - l/d) * (1 - 4 * l/d)
        
        y14 = TempFactor(S = S, l = 1, d = 4)
        c_y14 = 1 / 24 * (1 - l/d) * (1 - 2 * l/d)

        factT = (l/d)**(-3) * (c_yinf * yinf + c_y1 * y1 + c_y12 * y12 + c_y14 * y14)     
            
    if (echo): print("Temp Factor: " + str(factT))
    return(factT)

### Film Thickness Interpolating Function

In [49]:

# Normalized Minimum Film Thickness h0/c
#   Unitary inverse of Epsilon, eps = e/c for h0 = c - e ~~ h0/c = 1 - e/c ~~ (1 - h0/c) = eps
#   Alternatives: Infinite case by boolean, or else needing l & r for l/d ratio
#       l/d ratio not of given ratios 1, 1/2, 1/4 will be interpolated by EQ 12-16
def FilmThickness(S, r = None, l = None, inf = True, eps = False):
    d = 2*r
    if (inf):
        if (S < .002):
            print("Error in Thickness: Out of Graph Range")
            h = None
        elif (S < 0.1): #R^2 = .999 for S = 0.002 -> 0.4
            A = -.0303
            B = 13.7
            C = -78.2
            D = 199
            E = -185
            h = A + B * S + C * S**2 + D * S**3 + E * S**4
        elif (S < 0.5): #R^2 = .999 for S = 0.1 -> 0.6
            A = .368
            B = 5.33
            C = -19.5
            D = 32.2
            E = -19.5
            h = A + B * S + C * S**2 + D * S**3 + E * S**4
        elif (S < 1.5): #R^2 = .997 for S = 0.5 -> 2.0
            A = .904
            B = .147
            C = -.0729
            D = .0117
            E = 0
            h = A + B * S + C * S**2 + D * S**3 + E * S**4 
        elif (S <= 10): 
            h = 1
        
    elif (abs(l - d) < .001): # L/D = 1
        if (S < .002):
            print("Error in Thickness: Out of Graph Range")
            h = None
        elif (S < 0.06): #R^2 = .999 for S = 0.002 -> 0.1
            A = -8.31E-3
            B = 7.01
            C = -88.5
            D = 966
            E = -4235
            h = A + B * S + C * S**2 + D * S**3 + E * S**4
        elif (S < .6): #R^2 = .998 for S = 0.08 -> 0.8
            A = .0705
            B = 3.57
            C = 8.71
            D = 11.2
            E = -5.52
            h = A + B * S + C * S**2 + D * S**3 + E * S**4
        elif (S < 5): #R^2 = .999 for S = 0.8 -> 5
            A = .631
            B = .354
            C = -.154
            D = .0307
            E = -2.25E-3
            h = A + B * S + C * S**2 + D * S**3 + E * S**4
        elif (S < 10): #R^2 = 1 for S = 4 -> 10
            A = 0.75
            B = .125
            C = -.026
            D = 2.47E-3
            E = -8.71E-5
            h = A + B * S + C * S**2 + D * S**3 + E * S**4
        else: h = None
        return h
    #elif (abs(l / d - 1/2) < .001): # L/D = 1/2
    #    if (S <)
    #elif (abs(l / d - 1/4) < .001): # L/D = 1/4
    #    if

### Flow Factor Interpolating Function
Currently only for Infinite L/D Cases

In [50]:

# Flow Factor Q/(rcNl) calaculation
#   Fit values to readings from fig 12-19.
#   Non-infinite cases not implemented
def FlowFactor(S, inf = True):
    if(inf):
        if(S < 0.06):
            Q = 37.4 * S - 9.32E-3 # Linear Interpolation 0 -> 0.06, R^2 = 0.997
        elif(S < 0.341):
            Q = 3.59 + 0.456 * np.log(S) # Natural Log Interpolation 0.06 -> 0.4, R^2 = 0.916
        else: # S >= 0.341
            Q = 3.1
    else: Q = None
    return Q 

## Naturally-Fed Journal Bearing Iterative Solver

In [51]:
# -- Natural Journal Function Definition -- 
# Grade = Oil Grade, as a String ("10W-40") for 
def NaturalJournal(Grade, Tout_guess, T_in, load, rpm, l, r, c, convergence = 0.1, Metric = False, echo_iter = False, echo = False):
    p = load / (2 * r * l)
    rps = rpm /60 

    Tavg_guess = (T_in + Tout_guess)/2

    # Determining if it needs to be parsed as interpolated or Seireg for calculating mu
    #   Currently differentiated by Seireg being Int, Interpolation being String

    if(isinstance(Grade, str)): 
        try: 
            mu_guess = Visc_SAE_Inter(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            method = "Inter" # keep for later solving
        except: print("Likely could not Parse Grade")
    if(isinstance(Grade, int)):
        try: 
            mu_guess = Visc_SAE_Seireg(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            method = "Seireg" # keep for later solving
        except: print("Likely could not Parse Grade")
    
    S_guess = Sommerfeld(mu = mu_guess, r = r, c = c, N = rps, p = p)
    delT_guess = TempDelta(Tval = TempFactor(S = S_guess, l = l, d = 2*r, mu = mu_guess, N = rps, p = p, c = c), p = p, Metric = Metric)

    error = convergence*1.1 # Throwaway to run while loop
    n = 0
    while error > convergence:
        if(echo_iter): print(n)
        if(echo): 
            print("Tavg: " + str(Tavg_guess))
            print("Mu: " + str(mu_guess))
            print("Sommerfeld#: " + str(S_guess))
            print("Temp Delta: " + str(delT_guess))
        # Keeping track of iterative values
        Tavg_prev = Tavg_guess
        mu_prev = mu_guess
        S_prev = S_guess
        delT_prev = delT_guess
        n = n+1
        
        
        # Same as before, just iterating
        Tout_guess = T_in + delT_guess
        Tavg_guess = T_in + delT_guess/2
        match method:
            case "Inter": mu_guess = Visc_SAE_Inter(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            case "Seireg": mu_guess = Visc_SAE_Seireg(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
        S_guess = Sommerfeld(mu = mu_guess, r = r, c = c, N = rps, p = p)
        delT_guess = TempDelta(Tval = TempFactor(S = S_guess, l = l, d = 2*r, mu = mu_guess, N = rps, p = p, c = c), p = p, Metric = Metric)

        error = abs(delT_guess - delT_prev)

    print("Converged on Iteration:" + '\t' + str(n))
    print("Sommerfeld:" + '\t' + str(S_guess))
    print("Outlet Temperature:" + '\t' + str(T_in + delT_guess))
    return(S_guess, T_in + delT_guess)


## Pressure-fed Bearing Iterative Solver
**requires implementation of h_0 solver for finding eccentricity**

In [52]:
# -- Pressure Fed Journal Function Definition -- 
# Grade = Oil Grade, as a String ("10W-40") for 
def PressureJournal(Grade, Tout_guess, T_in, load, rpm, l, r, c, lg, ps, convergence = 0.1, Metric = False, echo_iter = False):
    rps = rpm /60 
    le = (l - lg)/2 # effective length of grooved
    p = load/2 / (2 * r * le) + ps # halved load, split length, + input pressure

    Tavg_guess = (T_in + Tout_guess)/2

    # Determining if it needs to be parsed as interpolated or Seireg for calculating mu
    #   Currently differentiated by Seireg being Int, Interpolation being String

    if(isinstance(Grade, str)): 
        try: 
            mu_guess = Visc_SAE_Inter(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            method = "Inter" # keep for later solving
        except: print("Likely could not Parse Grade")
    if(isinstance(Grade, int)):
        try: 
            mu_guess = Visc_SAE_Seireg(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            method = "Seireg" # keep for later solving
        except: print("Likely could not Parse Grade")
    
    S_guess = Sommerfeld(mu = mu_guess, r = r, c = c, N = rps, p = p)
    #delT_guess = TempDelta(Tval = TempFactor(S = S_guess, l = le, d = 2*r, mu = mu_guess, N = rps, p = p, c = c), p = p, Metric = Metric)
    delT_guess = np.pi * ps * r * c**3 / (3 * mu_guess * le) * (1 + 1.5 * )

    error = convergence*1.1 # Throwaway to run while loop
    n = 0
    while error > convergence:
        if(echo_iter): print(n)
        # Keeping track of iterative values
        Tavg_prev = Tavg_guess
        mu_prev = mu_guess
        S_prev = S_guess
        delT_prev = delT_guess
        n = n+1
        
        # Same as before, just iterating
        Tout_guess = T_in + delT_guess
        Tavg_guess = T_in + delT_guess/2
        match method:
            case "Inter": mu_guess = Visc_SAE_Inter(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            case "Seireg": mu_guess = Visc_SAE_Seireg(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
        S_guess = Sommerfeld(mu = mu_guess, r = r, c = c, N = rps, p = p)
        delT_guess = TempDelta(Tval = TempFactor(S = S_guess, l = le, d = 2*r, mu = mu_guess, N = rps, p = p, c = c), p = p, Metric = Metric)

        error = abs(delT_guess - delT_prev)

    print("Converged on Iteration:" + '\t' + str(n))
    print("Sommerfeld:" + '\t' + str(S_guess))
    print("Outlet Temperature:" + '\t' + str(T_in + delT_guess))
    return(S_guess, T_in + delT_guess)


SyntaxError: invalid syntax (3528974089.py, line 26)

# EMAE360 Testing

In [83]:
Grade = "10W-40" 
load_N = 54896.76 # testing
Tin_degC = 40
r_m = 28.027977E-3
l_m = 25.980451E-3
rpm = 8000
# cursed
rpm = rpm * np.pi


c = r_m*2/1000


Tout_g = 60

In [84]:
NaturalJournal(Grade = Grade, Tout_guess = Tout_g, T_in = Tin_degC, load = load_N, rpm = rpm, l = l_m, r = r_m, c = c, Metric = True, convergence=.01, echo_iter = True, echo = True)

0
Tavg: 50.0
Mu: 0.04383922465747996
Sommerfeld#: 0.12179044106540823
Temp Delta: 373.54560462187493
1
Tavg: 226.77280231093746
Mu: 0.0041450918948781066
Sommerfeld#: 0.01151554513288404
Temp Delta: 167.52374398961723
2
Tavg: 123.76187199480862
Mu: 0.010661555069632012
Sommerfeld#: 0.029619034198682448
Temp Delta: 194.35224341310908
3
Tavg: 137.17612170655454
Mu: 0.009080328909353529
Sommerfeld#: 0.025226204877700827
Temp Delta: 186.7757661912282
4
Tavg: 133.3878830956141
Mu: 0.009485812111715324
Sommerfeld#: 0.026352684153876423
Temp Delta: 188.69035214565258
5
Tavg: 134.3451760728263
Mu: 0.009380578701568576
Sommerfeld#: 0.02606033355833726
Temp Delta: 188.19133635166418
6
Tavg: 134.0956681758321
Mu: 0.009407821387174255
Sommerfeld#: 0.026136016892649053
Temp Delta: 188.3203825849992
7
Tavg: 134.1601912924996
Mu: 0.009400763952963624
Sommerfeld#: 0.026116410523421073
Temp Delta: 188.2869427751215
Converged on Iteration:	8
Sommerfeld:	0.02612148881349437
Outlet Temperature:	228.295603

(0.02612148881349437, 228.2956034695158)

Non-Pressure Fed Grooved Bearing

In [56]:
load_split = load_N/2
groove = l_m / 20 # UPDATE THIS
length_eff = (l_m - groove)/2

NaturalJournal(Grade = Grade_10W, Tout_guess = Tout_g, T_in = Tin_degC, load = load_split, rpm = rpm, l = length_eff, r = r_m, c = c, Metric = True, convergence=.01, echo_iter = True, echo = True)

0
Tavg: 50.0
Mu: 0.04383922465747996
Sommerfeld#: 0.036828746362113585
Temp Delta: 431.06345635512076
1
Tavg: 255.53172817756038
Mu: 0.0034406629686332623
Sommerfeld#: 0.002890454947124405
Temp Delta: -472.5844575373364
Outside of Interpolation acceptability range (Metric)


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

#### From 12-11 Pressure-fed Bearings

For a grooved journal bearing, must account for the journal length being reduced by 1/2 (by the split) as well as the by the actual axial length of the groove. NOTE that the load being accepted by this decreased length is also approximately halved, however -- that is, the pressure is split in half while the length accepted is also (approximately) halved, but the l/d ratio is (approximately) halved as well, which will cause more significant changes.

$l_{eff} = {(l_{tot} - l_{groove}) \over 2}$

For pressure-fed bearings:

- The pressure at the interface is equal to the normal pressure that would be generated plus the pumped pressure

- The heat generated is side flow that results is related to (beyond the natural parameters) the input pressure and the eccentricity of the shaft -- or more accurately, the eccentricity ratio of 

$\epsilon = {e \over c}$ where $h_0 = c - e$

such that

$Q_s = {\pi  p_s  r  c^3 \over 3  \mu  l_{eff}} \times (1 + 1.5 \times \epsilon^2)$

Along with this, the temperature rise becomes equal to:

$\Delta T = A \times {f r / c \over 1 + 1.5 \epsilon ^2} \times {S W^2 \over p_s r^4} $

Where A depends on the system:
- A = 0.0123 for Fahrenheit
- A = 978 10^6 for Celsius

True

# Script Testing 

EMAE370 Chapter 12: Journal Bearings

## Iterative Approach

In [57]:
Grd = 30 #SAE30

load_lbf = 10E3
rpm = 900
Tin_degF = 180

l_in = 12
r_in = 12

p_psi = load_lbf / (2*r_in * l_in)

c = r_in*2/1000

rps = 900 / 60

Tout_guess = 205
Tavg_guess = (180+205)/2

print("Pressure (psi): " + str(p_psi))

Pressure (psi): 34.72222222222222


In [58]:
Tavg_guess
print(c)

0.024


In [59]:
# Testing rational l/d case
NaturalJournal(Grade = Grd, Tout_guess = 205, T_in = Tin_degF, load = load_lbf, rpm = rpm, l = l_in, r = r_in, c = c)

# Testing interpolated l/d case
NaturalJournal(Grade = Grd, Tout_guess = 205, T_in = Tin_degF, load = load_lbf, rpm = rpm, l = l_in, r = r_in-.1, c = c)

# Testing Metric case
vals = NaturalJournal(Grade = Grd, Tout_guess = ConvertTemp(205), T_in = ConvertTemp(Tin_degF), load = 239401.3888888889, rpm = rpm, l = 0.30479999999999996, r = 0.30479999999999996, c = 0.0006096, Metric = True, convergence= .0001)
print("Equals deg F: " + str(ConvertTemp(vals[1], Metric=True)) + '\n')

# Testing Interpolation case
NaturalJournal(Grade = "30", Tout_guess = 205, T_in = Tin_degF, load = load_lbf, rpm = rpm, l = l_in, r = r_in, c = c)
# Testing 10W-40
NaturalJournal(Grade = "10W-40", Tout_guess = 205, T_in = Tin_degF, load = load_lbf, rpm = rpm, l = l_in, r = r_in, c = c)

# Note that

Converged on Iteration:	2
Sommerfeld:	0.20274919171300873
Outlet Temperature:	186.0465037493159
Converged on Iteration:	2
Sommerfeld:	0.19784152696857824
Outlet Temperature:	185.97898108603331
Converged on Iteration:	4
Sommerfeld:	0.035741734360170724
Outlet Temperature:	88.91126207056917
Equals deg F: 192.0402717270245

Converged on Iteration:	2
Sommerfeld:	0.17359991046717627
Outlet Temperature:	185.38090203908226
Converged on Iteration:	2
Sommerfeld:	0.3008371867568187
Outlet Temperature:	188.2846597721297


(0.3008371867568187, 188.2846597721297)

In [60]:

Tin_m = ConvertTemp(180, Metric = False)
#print(Tin_m)
Tout_m = ConvertTemp(205, Metric = False)
#print(Tout_m)

load_Pa = p_psi * 6894.76
print(load_Pa)

l_m = l_in * .0254
r_m = r_in * .0254
c_m = r_m * 2 /1000
print(l_m)
print(r_m)
#print(c_m)


mu = Visc_SAE_Seireg(Temp = (180+205)/2, Grade = 30, Metric = False)
mu_m = Visc_SAE_Seireg(Temp = (96.1 + 82.2)/2, Grade = 30, Metric = True)
#print(mu)
#print(mu_m)
#print(mu_m / 6890)
#mu = Visc_SAE_Inter(Temp = (180+205)/2, Grade = "30", Metric = False)
#mu_m = Visc_SAE_Inter(Temp = (96.1 + 82.2)/2, Grade = "30", Metric = True)
#print(mu_m / 6890)

S_guess = Sommerfeld(mu = mu, r = r_in, c = c, N = rps, p = p_psi)
print(S_guess)
S_guess = Sommerfeld(mu = mu_m, r = r_m, c = c_m, N = rps, p = load_Pa)
print(S_guess)

delT_g = TempDelta(Tval = TempFactor(S = S_guess, l = l_in, d = r_in*2, mu = mu, N = rps, p = p_psi, c =c), p = p_psi)
#print(TempFactor(S = S_guess, l = l_in, d = r_in*2, mu = mu, N = rps, p = p_psi, c =c))
print(delT_g)
delT_g = TempDelta(Tval = TempFactor(S = S_guess, l = l_m, d = r_m*2, mu = mu_m, N = rps, p = load_Pa, c = c_m), p = load_Pa, Metric = True)
#print(TempFactor(S = S_guess, l = l_m, d = r_m*2, mu = mu_m, N = rps, p = load_Pa, c = c_m))
#print(TempFactor(S = S_guess, l = l_in, d = r_in*2, mu = mu, N = rps, p = p_psi, c =c))
print(delT_g)

239401.3888888889
0.30479999999999996
0.30479999999999996
0.17260151016781655
0.17256751866691883
5.357324139179529
2.9857783546994976


In [61]:
ConvertTemp(88.9109, Metric = True)

192.03961999999999

In [62]:
mu = 1.5981621311834866e-06
r = 4
c = 0.008
N = 15
p = 312.5
(r/c)**2 * mu * N / p
#Sommerfeld(mu= 1.5981621311834866e-06, r = 4, c = .008, N = 15, p = 312.5)

0.01917794557420184

In [ ]:
# Guess Temperature
Tout_g = 205
Tavg_g = (Tin_degF + Tout_g)/2
print("Guess Film Average Temp, degF: " + str(Tavg_g))

In [ ]:
Tout = Tout_g

error = 1.1 # throwaway value
firstIterFlag = True
convergence_res = .1 # deg F
n = 1
while error > convergence_res:
    if firstIterFlag:   
        Tavg = (Tin_degF + Tout)/2
        mu = Visc_SAE_Seireg(Temp = Tavg, Grade = Grd, Metric = False)
        print("Viscosity: " + str(mu))
        S = Sommerfeld(mu = mu, r = r_in, c = c, N = rps, p = p_psi)
        delT = TempDelta(Tval = TempFactor(S = S, l = l_in, d = 2*r_in), p = p_psi, iter = n)
        firstIterFlag = False
    
    n = n+1
    # Storing for comparison to exit while loop
    Tavg_prev = Tavg
    mu_prev = mu 
    S_prev = S 
    delT_prev = delT 

    Tout = Tin_degF + delT 
    Tavg = Tin_degF + delT/2
    mu = Visc_SAE_Seireg(Temp = Tavg, Grade = Grd, Metric = False)
    S = Sommerfeld(mu = mu, r = r_in, c = c, N = rps, p = p_psi)
    delT = TempDelta(Tval = TempFactor(S = S, l = l_in, d = 2*r_in), p = p_psi, iter = n)

    error = abs(delT - delT_prev) 
    if (error <= convergence_res):
        print("S ~ last iteration Sommerfeld: " + str(S) + '\n' + "Tout ~ last iteration Temp: " + str(Tin_degF + delT))
        output = (S, Tin_degF + delT)



## Manual Approach

In [63]:
## SAE 30

load_lbf = 10E3
rpm = 900
Tin_degF = 180

l_in = 12
r_in = 12

# --- #

p_psi = load_lbf / (2*r_in * l_in)

c = r_in*2/1000
rps = 900 / 60

print("Pressure (psi): " + str(p_psi))

Pressure (psi): 34.72222222222222


In [64]:
Tout_1 = 205
Tavg_1 = (Tin_degF + Tout_1)/2
print("Guess Film Average Temp, degF: " + str(Tavg_1))

Guess Film Average Temp, degF: 192.5


In [65]:
mu_1 = 1.4E-6

In [66]:
S_1 = Sommerfeld(mu = mu_1, r = r_in, c = c, N = rps, p = p_psi)
print(S_1)

0.1512


In [67]:
delT_1 = TempDelta(Tval = TempFactor(S = S_1, l = l_in, d = 2*r_in), p = p_psi, iter = 1)

T_2 = Tin_degF + delT_1
print(T_2)

Temperature Delta (1): 4.869268232249714
184.8692682322497


In [68]:
Tavg_2 =  Tin_degF + delT_1/2
print("Guess Film Average Temp, degF: " + str(Tavg_2))

Guess Film Average Temp, degF: 182.43463411612487


In [69]:
mu_2 = 1.8E-6
S_2 = Sommerfeld(mu = mu_2, r = r_in, c = c, N = rps, p = p_psi)
delT_2 = TempDelta(Tval = TempFactor(S = S_2, l = l_in, d = 2*r_in), p = p_psi, iter = 2)
T_3 = Tin_degF + delT_2
print(T_3)

Temperature Delta (2): 5.8558786818167246
185.85587868181673


In [70]:
Tavg_3 =  Tin_degF + delT_2/2
print("Guess Film Average Temp, degF: " + str(Tavg_3))

Guess Film Average Temp, degF: 182.92793934090835


In [71]:
mu_3 = 1.8E-6
S_3 = Sommerfeld(mu = mu_3, r = r_in, c = c, N = rps, p = p_psi)
delT_3 = TempDelta(Tval = TempFactor(S = S_3, l = l_in, d = 2*r_in), p = p_psi, iter = 3)
T_4 = Tin_degF + delT_3

Temperature Delta (3): 5.8558786818167246


In [72]:
Tout = Tin_degF + delT_3
print(Tout)

185.85587868181673


In [73]:
h_0 = c * .8
print("h_0: " + str(h_0))
f = c/r_in * 1.4
print("f: " + str(f))
Q = 5.85 * r_in * c * rps * l_in
print("Qfull: " + str(Q))
Qs = .94 * Q 
print("Side Flow Qs: " + str(Qs))


h_0: 0.019200000000000002
f: 0.0028
Qfull: 303.264
Side Flow Qs: 285.06816
